# Four point functions in the Potts and $O(n)$ models

This notebook reproduces some known results about Potts and $O(n)$ four-point functions, using the new Julia code.

In [7]:
Pkg.activate(".") # activate the parent environment
using BootstrapVirasoro
if Threads.nthreads() == 1
    println("You are using a single thread. Consider starting julia with more threads, for instance by setting
    the environment variable `export JULIA_NUM_THREADS=auto`")
end

  Activating project at `~/Documents/Recherche/projet_these/code/BootstrapVirasoro/examples`


## Spectra

We create functions to generate the spectra of the $O(n)$ and Potts CFTs, with a cutoff:

\begin{align}
\mathcal{S}^{O(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in 2\mathbb{N}+1} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \frac12\mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^{PSU(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^\text{Potts} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}+2\\ s\in\frac{1}{r}\mathbb{Z}}} \bigcup  \left\{ V_{P_{(0,s)}}\right\}_{s\in \mathbb{N}+\frac12}\ .
\end{align}

## Bootstrap equations

We then numerically solve

\begin{align}
\sum_{V \in \mathcal{S}^{(s)}} D^{(s)}_V \mathcal I^{(s)}_V (x) = \sum_{V \in \mathcal{S}^{(t)}} D^{(t)}_V \mathcal I^{(t)}_V (x) = \sum_{V \in \mathcal{S}^{(u)}} D^{(u)}_V \mathcal I^{(u)}_V (x),
\end{align}

for some channel spectra $\mathcal{S}^{(s)}, \mathcal{S}^{(t)}, \mathcal{S}^{(u)}$, where $\mathcal I$ are interchiral conformal blocks, and the $D$'s are four-point structure constants.

We solve this system as
\begin{equation}
\underbrace{
\begin{bmatrix}
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [-\mathcal I^{(t)}_{V_j}(x_i)]_{ij} & [0] \\
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [0] & [-\mathcal I^{(u)}_{V_j}(x_i)]_{ij}
\end{bmatrix}}_A
\begin{bmatrix}
[D^{(s)}_{V_j}]_j \\
[D^{(t)}_{V_j}]_j \\
[D^{(u)}_{V_j}]_j
\end{bmatrix} = 
\begin{bmatrix}
\sum_{V_j \in \text{ known}} D^{(t)}_{V_j} \mathcal{I}^{(t)}_{V_j}(1-x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)\\
\sum_{V_j \in \text{ known}} D^{(u)}_{V_j} \mathcal{I}^{(u)}_{V_j}(1/x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)
\end{bmatrix}
\end{equation}
where the $x_i$ take more values than there are unknowns, i.e. $A$ is a tall rectangular matrix. 

To check numerical convergence, we solve two subsystems and compare the solutions. If the solutions are close, we know the computation has converged.


In [3]:
setprecision(BigFloat, 35, base=10)
c = CC(β=1/(big"0.8" + big"0.1"*im))
Δmax = 40.

function LoopFields(model)
    model === :On && return vcat(
        [Field(c, r=r, s=s) for r in 1:15 for s in -1+1//r:1//r:1],
        [Field(c, r=r, s=s) for r in 1//2:1:15 for s in -1+1//(2r):1//r:1 if (r*s)%1 == 0],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:2:15]
    )
    model === :PSUn && return vcat(
        [Field(c, r=r, s=s) for r in 1:15 for s in -1+1//r:1//r:1],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:15]
    )
    model === :Potts && return vcat(
        [Field(c, r=r, s=s) for r in 2:15 for s in -1+1//r:1//r:1],
        [Field(c, r=0, s=s, diagonal=true) for s in 1//2:1:3//2],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:15]
    )
end

LoopSpectrum(model, Δmax) = Spectrum(LoopFields(model), Δmax, interchiral=true);

function solve_bootstrap(indices, Δmax; signature = Dict(:s => 0, :t => 0, :u => 0), diags = Dict())
    fields = [Field(c, r=r, s=s) for (r, s) in indices]
    co = Correlation(fields..., Δmax=Δmax)
    SOn = LoopSpectrum(:On, Δmax)

    Schan_On = ChannelSpectra(
        co, SOn,
        signature=signature
    )

    for (chan, VP) in diags
        add!(Schan_On, VP)
    end

    b = BootstrapSystem(Schan_On)
    compute_linear_system!(b)
    solve!(b)
    return b
end

solve_bootstrap (generic function with 1 method)

## $\langle (\frac{1}{2}, 0)^2 (1, 0)^2 \rangle$

We expect a three-dimensional space of solutions. We can distinguish between them by choosing an appropriate signature.

In [4]:
indices = ((1 // 2, 0), (1 // 2, 0), (1, 0), (1, 0))
system1 = solve_bootstrap(indices, Δmax, signature=Dict(:s => 1, :t => 1//2, :u => 3//2))
system2 = solve_bootstrap(indices, Δmax, signature=Dict(:s => 0, :t => 3, :u => 3//2))
system3 = solve_bootstrap(indices, Δmax, signature=Dict(:s => 1, :t => 3//2, :u => 1//2));

MethodError: MethodError: no method matching ChannelSpectra(::BootstrapVirasoro.CorrelationNonChiral{Complex{BigFloat}}, ::BootstrapVirasoro.BulkSpectrum{Complex{BigFloat}}; signature::Dict{Symbol, Real})
This method may not support any kwargs.

Closest candidates are:
  ChannelSpectra(::Any, ::Spectrum{T}) where T got unsupported keyword argument "signature"
   @ BootstrapVirasoro ~/Documents/Recherche/projet_these/code/BootstrapVirasoro/src/spectra.jl:114


In [5]:
println(system.consts)
println(system2.consts)
println(system3.consts)

UndefVarError: UndefVarError: `system` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [ ]:
c = CC(β=1/big"0.8"+big"0.1"*im)
Δmax = 15
setprecision(BigFloat, 13)
indices = ((1 // 2, 0), (1 // 2, 0), (1, 0), (1, 0))
fields = [Field(c, r=r, s=s) for (r, s) in indices]
co = Correlation(fields..., Δmax=Δmax)
SOn = LoopSpectrum(:On, Δmax)

signatures = (s=1, t=1//2, u=3//2)
Schan_On = ChannelSpectra(co, SOn, signature=)

TaskFailedException: TaskFailedException

    nested task error: MethodError: no method matching length(::var"#68#71")
    The function `length` exists, but no method is defined for this combination of argument types.
    
    Closest candidates are:
      length(!Matched::DataStructures.IntSet)
       @ DataStructures ~/.julia/packages/DataStructures/IrAJn/src/int_set.jl:191
      length(!Matched::Cmd)
       @ Base process.jl:716
      length(!Matched::DataStructures.SparseIntSet)
       @ DataStructures ~/.julia/packages/DataStructures/IrAJn/src/sparse_int_set.jl:61
      ...
    
    Stacktrace:
     [1] length(g::Base.Generator{var"#68#71", BootstrapVirasoro.var"#316#320"{Field{Complex{BigFloat}}}})
       @ Base ./generator.jl:51
     [2] _similar_shape(itr::Base.Generator{var"#68#71", BootstrapVirasoro.var"#316#320"{Field{Complex{BigFloat}}}}, ::Base.HasLength)
       @ Base ./array.jl:663
     [3] collect(itr::Base.Generator{var"#68#71", BootstrapVirasoro.var"#316#320"{Field{Co

In [11]:
SOn

Degenerate:
<1, 1>
Non-diagonal:
(1//2, 0)
(1, 0), (1, 1)
(3//2, -2//3), (3//2, 0), (3//2, 2//3)
(2, -1//2), (2, 0), (2, 1//2), (2, 1)
(5//2, -4//5), (5//2, -2//5), (5//2, 0), (5//2, 2//5), (5//2, 4//5)
(3, -2//3), (3, -1//3), (3, 0), (3, 1//3), (3, 2//3), (3, 1)
(7//2, -6//7), (7//2, -4//7), (7//2, -2//7), (7//2, 0), (7//2, 2//7), (7//2, 4//7), (7//2, 6//7)
(4, -3//4), (4, -1//2), (4, -1//4), (4, 0), (4, 1//4), (4, 1//2), (4, 3//4), (4, 1)